In [ ]:
!pip install roboflow ultralytics opencv-python-headless scikit-image matplotlib seaborn

In [ ]:
from roboflow import Roboflow

rf = Roboflow(api_key="J1mmAuxLk77g4eiOvCmr")
project = rf.workspace("project-qemwo").project("mvtec-dl9fi")
version = project.version(5)
dataset = version.download("yolov8")

In [ ]:
import os
from PIL import Image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import yaml

# Define the base paths for each dataset type.
# These paths are given as an example; update them if your folder structure differs.
dataset_paths = {
    'train': '/content/mvtec-5/train/images',  # Adjust if needed
    'test': '/content/mvtec-5/valid/images',    # Adjust if needed
}

In [ ]:
num_samples = 8  # Number of images to sample per dataset

for dataset_name, image_folder in dataset_paths.items():
    if not os.path.isdir(image_folder):
        print(f"Directory {image_folder} does not exist.")
        continue
    image_files = [file for file in os.listdir(image_folder) if file.endswith('.jpg') or file.endswith('.png')]
    image_count = len(image_files)
    print(f"{dataset_name.capitalize()} dataset has {image_count} images.")
    if not image_files:
        print(f"No image files found in the {image_folder} directory.")
        continue
    interval = max(1, image_count // num_samples)
    selected_images = [image_files[i] for i in range(0, image_count, interval)][:num_samples]

    # Create a 2x4 grid to display the samples
    fig, axes = plt.subplots(2, 4, figsize=(20, 11))
    for ax, img_file in zip(axes.ravel(), selected_images):
        img_path = os.path.join(image_folder, img_file)
        image = Image.open(img_path)
        ax.imshow(image)
        ax.axis('off')
    plt.suptitle(f'Sample Images from {dataset_name.capitalize()} Dataset ({image_count} images)', fontsize=20)
    plt.tight_layout()
    plt.show()

In [ ]:
!pip install ultralytics
import ultralytics
ultralytics.checks()

In [ ]:
from ultralytics import YOLO

# Update this path if your data.yaml is stored elsewhere.
data_config = '/content/mvtec-5/data.yaml'
model = YOLO("yolov8n.pt")  # Using the nano version; change if necessary

results = model.train(
    data=data_config,  # Path to the dataset configuration file
    epochs=200,        # Number of epochs; adjust based on your needs
    imgsz=640,         # Image size input for training
    patience=50,       # Early stopping patience
    batch=16,          # Batch size (reduce if using CPU only)
    optimizer='auto',  # Optimizer configuration
    verbose=True       # Verbose output for each epoch
)

In [ ]:
import cv2
import matplotlib.pyplot as plt
from ultralytics import YOLO

# Load the trained model checkpoint.
# If you trained a new model, ensure the checkpoint file name/path matches.
model = YOLO('/content/runs/detect/train/weights/best.pt')  # Update with your actual model file
# Update the path to a sample image from your dataset.
image_path = '/content/mvtec-5/train/images/b10_jpg.rf.675dec7a95471b2d25b194fd3155ca98.jpg'

results = model.predict(
    image_path,
    conf=0.25,  # Confidence threshold
    iou=0.45,   # IOU threshold
    save=False, # Do not save the output image
    show=False  # Do not show using YOLO's built-in viewer
)

# Retrieve and display the image with predictions
img_with_predictions = results[0].plot()

plt.figure(figsize=(10, 10))
plt.imshow(cv2.cvtColor(img_with_predictions, cv2.COLOR_BGR2RGB))  # Convert BGR to RGB
plt.axis('off')
plt.title("YOLOv8 Prediction Result")
plt.show()

In [ ]:
import pandas as pd
import os
import matplotlib.pyplot as plt

# Define the path where your results.csv is saved.
# Ensure that post_training_files_path is defined. For example:
post_training_files_path = '/content/runs/detect/train/results.csv'  # Update if needed
results_csv_path = os.path.join(post_training_files_path, 'results.csv')

# Load the CSV file into a DataFrame
results_csv_path = "/content/runs/detect/train/results.csv"
df = pd.read_csv(results_csv_path)
df.columns = df.columns.str.strip()  # Remove any leading/trailing spaces

# Define a simple function to plot learning curves.
def plot_learning_curve(train_loss, val_loss, title):
    plt.figure(figsize=(10, 5))
    plt.plot(df[train_loss], label='Train Loss')
    plt.plot(df[val_loss], label='Validation Loss')
    plt.title(title)
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.legend()
    plt.grid(True)
    plt.show()

plot_learning_curve('train/box_loss', 'val/box_loss', 'Box Loss Learning Curve')
plot_learning_curve('train/cls_loss', 'val/cls_loss', 'Classification Loss Learning Curve')
plot_learning_curve('train/dfl_loss', 'val/dfl_loss', 'Distribution Focal Loss Learning Curve')